# Run length encoding

Run-length encoding (RLE) is a very simple form of lossless data compression in which runs of data (that is, sequences in which the same data value occurs in many consecutive data elements) are stored as a single data value and count, rather than as the original run. This is most useful on data that contains many such runs. Consider, for example, simple graphic images such as icons, line drawings, and animations. It is not useful with files that don't have many runs as it could greatly increase the file size

https://en.wikipedia.org/wiki/Run-length_encoding

In [ ]:
from PIL import Image
import numpy as np; print(np.__version__)

In [ ]:
logo = Image\
        .open('python-logo.png')\
        .convert(mode='P', palette=Image.ADAPTIVE, colors=5)
data = np.array(logo).reshape(-1)
print(data[0:200])
logo

# Pure python

In [ ]:
def rle_encode(sequence):
    """Run length encode array."""
    previous = sequence[0]
    count = 1
    out = []
    for element in sequence[1:]:
        if element == previous:
            count += 1
        else:
            out.append((count, previous))
            previous = element
            count = 1
    out.append((count, previous))
    return out

In [ ]:
rle_encode('abbbccccccc')

In [ ]:
rle_encode((1,1,1,1,2,2,2))

In [ ]:
def rle_decode(data):
    """Decode array with rle encoded data."""
    decoded = []
    for count, element in data:
        decoded.extend([element, ] * count)
    return decoded

In [ ]:
''.join(rle_decode([(1, 'a'), (3, 'b'), (7, 'c')]))

In [ ]:
import operator
timings = {}

def ratios(**new):
    assert len(new) == 1
    timings.update(**new)
    last = list(new.values())[0]
    print('\n'.join('%10s: %7.2f' % (name, t / last)
                    for name, t in sorted(timings.items(), key=operator.itemgetter(1))))

In [ ]:
%%timeit
rle_encode(data)

In [ ]:
ratios(python=5030)

# Numpy

In [ ]:
def rle_encode_numpy(sequence):
    diffs = np.concatenate(( np.array((True, )), np.diff(sequence)!=0))
    indices = np.concatenate((np.where(diffs)[0],np.array((sequence.size, ))))
    counts = np.diff(indices).astype('uint16')
    values = sequence[diffs].astype('uint8')
    return np.rec.fromarrays((counts, values),names=('count','value'))

In [ ]:
rle_encode_numpy(data)[0:5]

In [ ]:
%%timeit 
rle_encode_numpy(data)

In [ ]:
ratios(numpy=144)

# Cython

Cython is a superset of the Python programming language, designed to give C-like performance with code that is mostly written in Python.

https://en.wikipedia.org/wiki/Cython

In [ ]:
import cython; print(cython.__version__)
%load_ext cython

In [ ]:
%%cython -a
def rle_encode_cython(sequence):
    """Run length encode array."""
    previous = sequence[0]
    count = 1
    out = []
    for element in sequence[1:]:
        if element == previous:
            count += 1
        else:
            out.append((count, previous))
            previous = element
            count = 1
    out.append((count, previous))
    return out

In [ ]:
rle_encode_cython(data)[0:5]

In [ ]:
%%timeit 
rle_encode_cython(data)

In [ ]:
ratios(cython=3280)

In [ ]:
%%cython -a
cimport numpy as np
cimport cython

def rle_encode_cython_types(np.uint8_t [:] sequence not None):
    """Run length encode array."""
    cdef unsigned long i, n, count = 1, size = sequence.size
    cdef unsigned char element, previous = sequence[0]
    out = []
    for i in range(1,size):
        element = sequence[i]
        if element == previous:
            count += 1
        else:
            out.append((count, previous))
            previous = element
            count = 1
    out.append((count, previous))
    return out

In [ ]:
%%timeit 
rle_encode_cython_types(data)

In [ ]:
ratios(cython_types=71)

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython

data_type = np.uint8
ctypedef np.uint8_t data_type_t
    
@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)

def rle_encode_cython_unsafe(data_type_t [:] sequence):
    """Run length encode array."""
    cdef unsigned long i, n=0, count = 1, size = 1000
    cdef unsigned char element
    cdef unsigned char previous = sequence[0]

    counts = np.empty(size,dtype=np.uint16)
    values = np.empty(size,dtype=np.uint8)
    
    for i in range(1,size):
        element = sequence[i]
        if element == previous:
            count += 1
        else:
            counts[n] = count
            values[n] = previous
            previous = element
            count = 1
            n += 1
          
    counts[n] = count
    values[n] = previous
    n += 1  
    return (counts[:n], values[:n])
 

In [ ]:
counts, values = rle_encode_cython_unsafe(data)
print(counts[:5],values[:5])

In [ ]:
%%timeit 
rle_encode_cython_unsafe(data)

In [ ]:
ratios(cython_unsafe=8)

# Numba
Using [Just-in-time compilation](https://en.wikipedia.org/wiki/Just-in-time_compilation)

In [ ]:
import numba; print(numba.__version__)

In [ ]:
@numba.jit
def rle_encode_numba(sequence):
    """Run length encode array."""
    previous = sequence[0]
    count = 1
    out = []
    for element in sequence[1:]:
        if element == previous:
            count += 1
        else:
            out.append((count, previous))
            previous = element
            count = 1
    out.append((count, previous))
    return out

In [ ]:
%%time
a = rle_encode_numba(data)

In [ ]:
%%time
a = rle_encode_numba(data.astype(np.int32))

In [ ]:
rle_encode_numba(data)[0:5]

In [ ]:
%%timeit 
rle_encode_numba(data)

In [ ]:
ratios(numba=80)